# Título: Automatización de Soporte TI con Gemini Vision y Trello

## Objetivo:
Desarrollar una solución automatizada que utilice videos de problemas reportados por usuarios para crear, resumir, etiquetar y gestionar tarjetas en Trello de manera automática, aprovechando las capacidades de Gemini Pro Vision.

## Instrucciones:
Sigue los pasos detallados en este notebook para configurar el entorno, autenticarte en Google Cloud Platform, procesar videos con modelos de IA generativa, y finalmente, interactuar con la API de Trello para gestionar tickets de soporte.

### Instalamos dependencias

In [ ]:
# Instalamos el cliente de Python para Vertex AI
!pip install google-api-python-client

In [ ]:
# Instalamos paquetes para conectarnos a los servicios de Google
!pip install google-auth
!pip install google-auth-oauthlib

In [ ]:
# Instalamos Vertex AI (Plataforma de desarrollo de soluciones de AI Generativa).
# Con esta plataforma podremos acceder a los differentes modelos de Gemini de Google y a otros servicios de Agentes y Chat.
!pip install vertexai

In [ ]:
# Instala Vertex Search
!pip install --upgrade --quiet google-cloud-discoveryengine

In [ ]:
# Instalamos LangChain y las dependencias de Google Vertex
!pip install langchain-google-vertexai
!pip install langchain

### Nos autenticamos para acceder a Google Cloud.

In [19]:
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

Importamos Vertex AI, y desde Vertex AI obtenemos los constructores de las clases GenerativeModel y Part, que nos permitiran interactuar fácilmente con la API y diferentes tipos de formatos con los que podremos interactuar con el modelo (Texto, Imágenes, Audio, etc.)

In [20]:
import vertexai
from vertexai.generative_models import GenerativeModel, Part

Cargamos el video de ejemplo

In [ ]:
import IPython

video_url = "https://storage.googleapis.com/workshop-bucket-elgueta-2024/Screen%20Recording%202024-04-26%20at%2014.35.40.mov"


IPython.display.Video(video_url, width=450)

Ten cuidado de indicar el project_id correcto.

In [ ]:
def generate_video_description(project_id, location, model_name, video_file_uri, prompt):
    vertexai.init(project=project_id, location=location)

    model = GenerativeModel(model_name)

    video_file = Part.from_uri(video_file_uri, mime_type="video/mov")

    contents = [video_file, prompt]

    response = model.generate_content(contents)
    return response.text

# Configuramos los parametros de la API para que funcione con nuestro proyecto de Google Cloud
project_id = "prj-uc-genai-labs"
location = "us-central1"

# Configuramos los parametros del modelo y el tipo de input que le entregarémos.
model_name = "gemini-1.5-pro-preview-0409"
video_file_uri = "gs://workshop-bucket-elgueta-2024/Screen Recording 2024-04-26 at 14.35.40.mov"

prompt = """
  Provide a description of the video.
  The description should also contain anything important which people say in the video.
  Output a json with this schema:
  {
    "description": "The description of the problem the user is facing",
    }
"""

description = generate_video_description(project_id, location, model_name, video_file_uri, prompt)

# Pequeño Debug para ver que nos está devolviendo la API
print(description)

## Integración con Trello para function calling
Con la configuración completa de Google Cloud, Vertex AI, y los parámetros del modelo, nuestro siguiente paso es integrar la API de Trello para permitir que Gemini administre tickets. Los pasos a seguir son:
#
1. Registrarse en Trello visitando https://www.trello.com.
2. Crear un Power-Up en Trello, que actuará como un puente entre nuestro tablero de Trello y su API.
3. Obtener y guardar la API Key y el Token de Trello para permitir la interacción con la plataforma sin necesidad de usar credenciales directas.
4. Localizar el idList de nuestro tablero de Trello siguiendo las instrucciones disponibles en: [Cómo encontrar el idList de un tablero](https://docs.n8n.io/integrations/builtin/app-nodes/n8n-nodes-base.trello/#how-do-i-find-the-list-id).


In [31]:
trello_id_list = "tu-trello-id-list"
trello_api_key = "tu-trello-api-key"
trello_token =  "tu-trello-token"

Primero definimos la función que nos permitirá crear una tarjeta en Trello.

In [24]:
# Esta función crea una tarjeta en Trello utilizando la API de Trello.
def create_trello_card(name, due, start):
    # Importamos los módulos necesarios para realizar la solicitud HTTP y manejar el formato JSON.
    import requests
    import json

    # URL de la API de Trello para crear una nueva tarjeta.
    url = "https://api.trello.com/1/cards"

    # Definimos los headers de la solicitud, especificando que esperamos una respuesta en formato JSON.
    headers = {"Accept": "application/json"}

    # Parámetros de la solicitud, incluyendo el ID de la lista de Trello, la API key, el token, y los datos de la tarjeta.
    query = {
        "idList": trello_id_list,  # ID de la lista en Trello donde se creará la tarjeta.
        "key": trello_api_key,     # Clave de la API de Trello.
        "token": trello_token,     # Token de acceso a la API de Trello.
        "name": name,              # Nombre de la tarjeta.
        "due": due,                # Fecha de vencimiento de la tarjeta.
        "start": start,            # Fecha de inicio de la tarjeta (no siempre es soportado por Trello).
    }

    # Realizamos la solicitud POST a la API de Trello para crear la tarjeta.
    response = requests.request("POST", url, headers=headers, params=query)

    # Imprimimos la respuesta de la API de Trello, formateada en JSON para una mejor lectura.
    print(
        json.dumps(
            json.loads(response.text), sort_keys=True, indent=4, separators=(",", ": ")
        )
    )


## Creando el Search App para hacer RAG

En este paso, insertaremos un dataset que incluya la documentación de LangChain en Vertex Search, la cual los vectorizará por nosotros e insertará en la base de datos de vectores, a través de la cual podremos hacer búsqueda semántica.

1.   Descargamos el dataset en formato json desde [aquí](https://huggingface.co/datasets/LangChainDatasets/langchain-howto-queries).
2.   Seguimos estas [instrucciones](https://cloud.google.com/generative-ai-app-builder/docs/try-enterprise-search#create_and_preview_a_search_app_for_structured_data_from) para crear una Search App.
3. Creamos la función para poder hacer búsquedas con Vertex Search.


In [32]:
from langchain_community.retrievers import (
    GoogleVertexAISearchRetriever,
)
# Definimos el ID del almacén de datos donde se realizará la búsqueda
data_store_id = "tu-data-store-id"

# Función para recuperar información de una búsqueda utilizando Vertex AI Search
def retrieve_from_search(query):
    # Definimos el ID del proyecto en Google Cloud
    PROJECT_ID = project_id
    # Definimos la ubicación del almacén de datos
    LOCATION_ID = "global"
    # Definimos el ID del almacén de datos
    DATA_STORE_ID = data_store_id
    # Creamos una instancia del retriever de Vertex AI Search con los parámetros definidos
    retriever = GoogleVertexAISearchRetriever(
        project_id=PROJECT_ID,
        location_id=LOCATION_ID,
        data_store_id=DATA_STORE_ID,
        max_documents=3,  # Número máximo de documentos a recuperar
        engine_data_type=1,  # Tipo de datos del motor de búsqueda
    )
    # Invocamos el retriever con la consulta proporcionada y almacenamos el resultado
    result = retriever.invoke(query)
    # Devolvemos el resultado de la búsqueda
    return result

Creamos un modelo Pydantic para nuestras funciones, lo que facilitará que LangChain comprenda la estructura de dichas funciones, y se las entregue a Gemini.



In [26]:
from pydantic import BaseModel, Field
from typing import Optional
from datetime import datetime


class CreateTrelloCardandAnswer(BaseModel):
    """
    Function that calls the Trello API to create a new card.

    """
    name: str = Field(..., description="Short description of the user problem")
    due: Optional[datetime] = Field(...,description="Name of the card")
    start: Optional[datetime] = Field(...,description="Name of the card")
    query: str = Field(..., description="Query to search")

Haremos un segundo llamado al LLM, para que en base a la descripción del video, extraiga los argumentos de nuestras funciones.

In [27]:
# Importamos la integración de LangChain con Vertex AI
from langchain_google_vertexai import ChatVertexAI
from langchain_core.prompts import ChatPromptTemplate

Convertimos la funciób CreateTrelloCardandAnswer a una herramienta de LangChain, y luego hacemos un llamado a Gemini, que usando function calling, nos devolverá un json que indicará que herramientas deben usarse.

In [ ]:
initial_prompt = """
Output json. Call the Trello Tool to create a new ticket and the RetrieveSearch to find the answer.
User input: {input}
"""
tools = [CreateTrelloCardandAnswer]
llm = ChatVertexAI(
    model="gemini-1.5-pro-preview-0409",
    project_id="tu-project-id",
    location="us-central1",
)

prompt = ChatPromptTemplate.from_template(initial_prompt)

llm_with_tools = llm.bind_tools(tools)
chain = prompt | llm_with_tools

result = chain.invoke({"input": description})

print(result.additional_kwargs)

Una vez obtenemos el resultado, debemos parsearlo para obtener el nombre y argumentos de las funciones que debe ser llamadas. Una vez obtenido esto, llamamos a la función, que hará 2 cosas;
1. Crear el ticket en Trello.
2. Buscar en Vertex Search la documentación apropiada para contestarle al usuario.

In [ ]:
from langchain_core.messages.base import message_to_dict
import json

response_dict = message_to_dict(result)
function_call = response_dict.get("data", {}).get("additional_kwargs", {}).get("function_call", {})


# Assuming 'arguments' is a JSON string, we need to parse it
arguments_str = function_call.get("arguments", "{}")  # Default to empty JSON object if not found
try:
    arguments = json.loads(arguments_str)  # Attempt to parse the string as JSON
except json.JSONDecodeError:
    arguments = {}  # If parsing fails, default to an empty dictionary


if function_call.get("name") == "CreateTrelloCardandAnswer":
    print(type(arguments))  # This should now be 'dict' if the JSON was parsed successfully
    create_trello_card(
        name=arguments.get("name"),
        due=arguments.get("due"),
        start=arguments.get("start"),
    )
    answer = retrieve_from_search(arguments.get("query"))

print(answer)

Ahora con la respuestas de Vertex Search, podemos hacer un último llamado a Gemini, adjuntando la documentación en el prompt, para que le conteste al usuario y resuelva su problema.

In [ ]:
initial_prompt = """
Based on the user input, and the content retrieved from the search engine, output a response that resolves the user problem.
User input: {input}
Search result: {search_result}
"""
llm = ChatVertexAI(
    model="gemini-1.5-pro-preview-0409",
    project_id="prj-uc-genai-labs",
    location="us-central1",
)

prompt = ChatPromptTemplate.from_template(initial_prompt)

chain = prompt | llm

result = chain.invoke({"input": description, "search_result": answer})

message_to_dict(result)